In [44]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import ensemble
from xgboost import XGBClassifier
from catboost import CatBoostClassifier, Pool
from catboost.utils import get_confusion_matrix

In [4]:
# !pip install xgboost
# !pip install catboost

     -------------------------------------- 89.1/89.1 MB 454.9 kB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
df = pd.read_csv('AirPass.csv')
df.head()

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied


In [6]:
df.isnull().sum().sum()

310

In [7]:
df['Arrival Delay in Minutes'] = df['Arrival Delay in Minutes'].fillna(df['Arrival Delay in Minutes'].median())
df['Arrival Delay in Minutes'].mean()

15.133392362180475

In [8]:
df.groupby('Gender')['satisfaction'].value_counts()*100/df.shape[0]

Gender  satisfaction           
Female  neutral or dissatisfied    29.058554
        satisfied                  21.687327
Male    neutral or dissatisfied    27.608177
        satisfied                  21.645942
Name: satisfaction, dtype: float64

In [9]:
df.groupby('Type of Travel')['satisfaction'].value_counts()*100/df.shape[0]

Type of Travel   satisfaction           
Business travel  satisfied                  40.177472
                 neutral or dissatisfied    28.785225
Personal Travel  neutral or dissatisfied    27.881506
                 satisfied                   3.155798
Name: satisfaction, dtype: float64

In [10]:
df.groupby('Class')['satisfaction'].value_counts()*100/df.shape[0]

Class     satisfaction           
Business  satisfied                  33.184478
          neutral or dissatisfied    14.614452
Eco       neutral or dissatisfied    36.614567
          satisfied                   8.374076
Eco Plus  neutral or dissatisfied     5.437712
          satisfied                   1.774715
Name: satisfaction, dtype: float64

In [11]:
df['satisfaction'] = df['satisfaction'].map({'neutral or dissatisfied':0 , 'satisfied':1})
df['Customer Type'] = df['Customer Type'].map({'Loyal Customer':1, 'disloyal Customer':0})
df['Type of Travel'] = df['Type of Travel'].map({'Personal Travel':0, 'Business travel':1})
df['Gender'] = df['Gender'].map({'Male': 0, 'Female': 1})

In [12]:
df=pd.get_dummies(df)
df.shape

(103904, 27)

In [19]:
df['']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103904 entries, 0 to 103903
Data columns (total 27 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Unnamed: 0                         103904 non-null  int64  
 1   id                                 103904 non-null  int64  
 2   Gender                             103904 non-null  int64  
 3   Customer Type                      103904 non-null  int64  
 4   Age                                103904 non-null  int64  
 5   Type of Travel                     103904 non-null  int64  
 6   Flight Distance                    103904 non-null  int64  
 7   Inflight wifi service              103904 non-null  int64  
 8   Departure/Arrival time convenient  103904 non-null  int64  
 9   Ease of Online booking             103904 non-null  int64  
 10  Gate location                      103904 non-null  int64  
 11  Food and drink                     1039

In [20]:
X = df.drop('satisfaction', axis=1)
y = df['satisfaction']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=26)
y_test.shape

(20781,)

In [23]:
scaler = preprocessing.StandardScaler()
scaler.fit(X_train) 
X_train = scaler.transform(X_train) 
X_test = scaler.transform(X_test) 
X_test[0][0]

1.193295224074621

In [25]:
model_lr = linear_model.LogisticRegression()
model_lr.fit(X_train, y_train)
preds_test = model_lr.predict(X_test)
metrics.f1_score(preds_test, y_test)

0.8550407188991856

In [29]:
model_ada = ensemble.AdaBoostClassifier(tree.DecisionTreeClassifier(random_state=26),random_state=26,learning_rate=0.01)

model_ada.fit(X_train, y_train)
preds_test = model_ada.predict(X_test)
metrics.f1_score(preds_test, y_test)

0.9376063216813916

In [34]:
model_for_gs = ensemble.GradientBoostingClassifier()
params = {"n_estimators":2**np.arange(8), "learning_rate":0.1**np.arange(3)}
gs = model_selection.GridSearchCV(model_for_gs, 
                  params, 
                  cv=3, 
                  scoring='f1',
                  verbose=5)
 
gs.fit(X_train, y_train)
 
print("Лучшие гиперпараметры:", gs.best_params_)
print("Лучшее значение метрики:", gs.best_score_)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV 1/3] END .learning_rate=1.0, n_estimators=1;, score=0.873 total time=   0.9s
[CV 2/3] END .learning_rate=1.0, n_estimators=1;, score=0.870 total time=   0.4s
[CV 3/3] END .learning_rate=1.0, n_estimators=1;, score=0.871 total time=   0.4s
[CV 1/3] END .learning_rate=1.0, n_estimators=2;, score=0.880 total time=   1.2s
[CV 2/3] END .learning_rate=1.0, n_estimators=2;, score=0.878 total time=   0.9s
[CV 3/3] END .learning_rate=1.0, n_estimators=2;, score=0.875 total time=   0.7s
[CV 1/3] END .learning_rate=1.0, n_estimators=4;, score=0.901 total time=   1.4s
[CV 2/3] END .learning_rate=1.0, n_estimators=4;, score=0.896 total time=   1.6s
[CV 3/3] END .learning_rate=1.0, n_estimators=4;, score=0.897 total time=   1.4s
[CV 1/3] END .learning_rate=1.0, n_estimators=8;, score=0.920 total time=   3.0s
[CV 2/3] END .learning_rate=1.0, n_estimators=8;, score=0.920 total time=   2.2s
[CV 3/3] END .learning_rate=1.0, n_estimators=8;

In [36]:
model_xgb = XGBClassifier(random_state=26)
model_xgb.fit(X_train,y_train)
preds_test = model_xgb.predict(X_test)
metrics.f1_score(preds_test, y_test)

0.9581571333184955

In [37]:
model = CatBoostClassifier(random_state=26)
model.fit(X_train, y_train)
preds_class = model.predict(X_test)
metrics.f1_score(preds_class, y_test)

Learning rate set to 0.068023
0:	learn: 0.6018110	total: 304ms	remaining: 5m 3s
1:	learn: 0.5117834	total: 525ms	remaining: 4m 22s
2:	learn: 0.4550707	total: 734ms	remaining: 4m 3s
3:	learn: 0.4109974	total: 951ms	remaining: 3m 56s
4:	learn: 0.3586388	total: 1.19s	remaining: 3m 57s
5:	learn: 0.3283911	total: 1.4s	remaining: 3m 51s
6:	learn: 0.3068413	total: 1.54s	remaining: 3m 38s
7:	learn: 0.2872780	total: 1.67s	remaining: 3m 27s
8:	learn: 0.2702377	total: 1.8s	remaining: 3m 18s
9:	learn: 0.2568038	total: 1.97s	remaining: 3m 15s
10:	learn: 0.2460249	total: 2.09s	remaining: 3m 7s
11:	learn: 0.2371699	total: 2.21s	remaining: 3m 2s
12:	learn: 0.2227433	total: 2.33s	remaining: 2m 57s
13:	learn: 0.2113301	total: 2.5s	remaining: 2m 56s
14:	learn: 0.2019340	total: 2.62s	remaining: 2m 52s
15:	learn: 0.1969028	total: 2.72s	remaining: 2m 47s
16:	learn: 0.1919290	total: 2.88s	remaining: 2m 46s
17:	learn: 0.1882537	total: 3.03s	remaining: 2m 45s
18:	learn: 0.1816900	total: 3.21s	remaining: 2m 45s

0.9599288612238093

In [45]:
cm = get_confusion_matrix(model, Pool(X_train, y_train))

In [46]:
print(cm)

[[46697.   509.]
 [ 1230. 34687.]]


In [48]:
pd.DataFrame(
    {
        "feature_importance": model.get_feature_importance(),
        "feature_names": df.drop(columns="satisfaction").columns,
    }
).sort_values(by=["feature_importance"], ascending=False)

,feature_importance,feature_names
7,24.561925,Inflight wifi service
5,19.814764,Type of Travel
12,7.624434,Online boarding
3,6.976861,Customer Type
23,5.261931,Class_Business
4,3.446749,Age
17,3.395929,Baggage handling
18,3.303830,Checkin service
10,3.298033,Gate location
13,3.122622,Seat comfort
